# Projeto Final IA

## Etapa 1

### Setup | Dependências

In [6]:
from pandas import read_excel

### Loading Dataset

In [7]:
from pathlib import Path
fp = "../data/MotorUniversal_2k_5kHz_SemNorm.xlsx"

with Path(fp).open("r", encoding="latin-1") as fd:
    data = read_excel(fd)


BadZipFile: File is not a zip file